In [3]:
import sys
import os
from datetime import datetime, timedelta
sys.path.append(os.path.split(os.path.split(sys.path[0])[0])[0])

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np
import plotly_template as pt
from functions import emissions, func, mp_funcs, speed
import definitions
import pandas as pd
import pickle
import math
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

# pio.renderers.default = "browser"

# Generating Iterations of the Heatmaps

### Creating the emissions plot function

### File locations and constants

In [5]:
DATA_FOLDER =['2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14']
DATA_DIR = r'Z:\6 ACTION Project\SUMO\Sim_Output_Data\8-01-2020'
# DATA_DIR_FULL_PATH = os.path.join(definitions.DATA_DIR, DATA_FOLDER)
# DATA_DIR_FULL_PATH = os.path.join(DATA_DIR, DATA_FOLDER)
RAW_DATA_FILE = 'data.csv'
RAW_DATA_SUMMARY = 'data_summary.csv'
RAW_DATA_FILE_PARQUET = 'data.parquet'
RAW_DATA_SUMMARY_PARQUET = 'data_summary.parquet'

## Read in the Data

In [9]:
raw_data = {}
summary_data = {}
days_extended = 1
start_hour = "23:00:00.000"
for folder in DATA_FOLDER:
    DATA_DIR_FULL_PATH = os.path.join(DATA_DIR, folder)
    try:
        raw_data[folder] = pd.read_parquet(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE_PARQUET),)
        summary_data[folder] = pd.read_parquet(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_SUMMARY_PARQUET),)

    except (FileNotFoundError, OSError):
        
        date_low = (datetime.strptime(folder, "%Y-%m-%d") - timedelta(days=days_extended)).strftime("%m/%d/%Y")
        start_time = " ".join([date_low, start_hour])
        raw_data[folder] = pd.read_csv(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE), low_memory=False, index_col=0)
        summary_data[folder] = pd.read_csv(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_SUMMARY), header=[0, 1], index_col=0)
        try:
            raw_data[folder].loc[:, 'timestep_time'] = mp_funcs.apply_by_multiprocessing(raw_data[folder]['timestep_time'], func, start_time=start_time)
        except TypeError:
            pass
        raw_data[folder].to_parquet(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE_PARQUET))
        table = pa.Table.from_pandas(summary_data[folder])
        pq.write_table(table, os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_SUMMARY_PARQUET))
        # summary_data[folder].to_parquet(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_SUMMARY_PARQUET), index=True)

In [10]:
for folder in DATA_FOLDER:
    raw_data[folder]['timestep_time'] = pd.to_datetime(raw_data[folder]['timestep_time'])

### Bining the Emissions Data

In [11]:
resample_period = '120T' # aka the bin size
resample_period_text = '120 min'
bin_size = 5

In [18]:
# df = pd.read_csv(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE), low_memory=False, index_col=0)
binned_emissions_dict = {}
for folder in DATA_FOLDER:
    # raw_data[folder]['timestep_time'] = pd.to_datetime(raw_data[folder]['timestep_time'])
    emissions.bin_2D_sum(emissions_df=raw_data[folder], bin_column=['vehicle_fuel'], resample_period=resample_period, bin_size=5, processor_num=2)
    binned_emissions_dict[folder] = pickle.load(open(os.path.join(definitions.DATA_DIR, 'emissions_dict.pkl'), 'rb'))

func:bin_2D_sum took: 6.080161809921265 sec
func:bin_2D_sum took: 4.9635045528411865 sec
func:bin_2D_sum took: 5.830761909484863 sec
func:bin_2D_sum took: 6.06493067741394 sec
func:bin_2D_sum took: 6.158908367156982 sec


## Plotting the Data

#### Plot Constants

In [19]:
plot_radius = 8
mapbox_key = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"

colorscale = [
              [0.0, "rgba(0, 255, 204, 0)"],
              [0.2, "rgb(0, 255, 51)"],
              [0.4, "rgb(204, 255, 0)"],
              [0.6, "rgb(255, 204, 51)"],
              [0.8, "rgb(255, 102, 51)"],
              [1.0, "rgb(204,0,0)"],
            ]

color_axis = dict(
            cmin=0,
            cmax=None,
            showscale=True,
            colorscale=colorscale,
            colorbar=dict(
                outlinecolor="black",
                outlinewidth=2,
                ticks="outside",
                # tickfont=pt.font_dict,
                tickvals=None, # tickvalues,
                ticktext=None,
                title=f"[gal per {resample_period_text}]<br /> <br />",
                tickformat="e",
                # titlefont=pt.font_dict,
            )
)

In [22]:
binned_emissions_dict.keys()

dict_keys(['2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14'])

In [28]:
time = '15:00'
plot_time_list = [['2020-02-10', pd.to_datetime('2020-02-10T' + time)], ['2020-02-11', pd.to_datetime('2020-02-11T' + time)], ['2020-02-12', pd.to_datetime('2020-02-12T' + time)], 
                 ['2020-02-13', pd.to_datetime('2020-02-13T' + time)], ['2020-02-14', pd.to_datetime('2020-02-14T' + time)]]
plot_titles = ['2020-02-10', '2020-02-10','2020-02-10','2020-02-10', '2020-02-10']

plot_titles = [time[0] + f"<br /> Total Fuel Consumed: {round(binned_emissions_dict[time[0]][time[1]]['interval_sum'], 2)} gallons" for i, time in enumerate(plot_time_list)]

num_plots = len(plot_time_list)
fig = make_subplots(rows=1, cols=num_plots,
                    specs=[[{'type': 'mapbox', }] * num_plots],
                    subplot_titles=plot_titles,
                    column_widths=[(1 - 0.1 * (num_plots / 2)) / num_plots]*num_plots,
                    horizontal_spacing=0.05,)

mapbox_dict = {'mapbox': dict(accesstoken=mapbox_key,
                                bearing=0,
                                style='mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte',
                                center=go.layout.mapbox.Center(
                                    lat=33.126275,
                                    lon=-87.54891
                                ),
                                pitch=0,
                                zoom=14.45),}

max_value = -20
for i, time in enumerate(plot_time_list):
    
    z_data = binned_emissions_dict[time[0]][time[1]]['data']
    lat_edges = binned_emissions_dict[time[0]][time[1]]['lat']
    lon_edges = binned_emissions_dict[time[0]][time[1]]['lon']
    max_value = binned_emissions_dict[time[0]][time[1]]['max_value'] if binned_emissions_dict[time[0]][time[1]]['max_value'] > max_value else max_value
    
    fig.add_trace(go.Densitymapbox(lat=lat_edges,
                                    lon=lon_edges,
                                    z=z_data,
                                    # customdata=summed_array[0][2],
                                    # hovertemplate='%{customdata}' if sum else None,
                                    radius=plot_radius,
                                    hoverinfo='z',
                                    coloraxis="coloraxis",
                                    ),
                    row=1,
                    col=i+1)
    
    mapbox_dict['mapbox'+ str(i+2)] =  mapbox_dict['mapbox']
    
color_axis['cmax'] = math.ceil(max_value * 1000) / 1000

len_of_max = len(str(color_axis['cmax']))
# tickvalues = np.around(np.linspace(0, np.round(color_axis['cmax'], len_of_max-1), 7), 3)
tickvalues = np.around(np.linspace(0, np.round(color_axis['cmax'], len_of_max), 7), 3)
ticktext = [str(val) for val in tickvalues]

color_axis['colorbar']['tickvals'] = tickvalues
color_axis['colorbar']['ticktext'] = ticktext

In [29]:
fig.update_layout(mapbox_dict)

fig.update_layout(
    title={"text": "Fuel Consumption", 'yanchor': 'middle', 'xanchor': 'auto'},
    hovermode='closest',
    coloraxis=color_axis
)

fig.show('browser')
# pio.write_json(fig, os.path.join(definitions.ROOT, 'raw_plots', 'mapbox.plotly'))

## Generating an average speed heatmap

In [78]:
average_speed_dict = {}
for folder in DATA_FOLDER:
    speed.bin_2D_average(emissions_df=raw_data[folder], bin_column=['vehicle_speed'], resample_period=resample_period, bin_size=5, processor_num=8, 
                                                      output_path=os.path.join(definitions.DATA_DIR, 'emissions_dict.pkl'))
    average_speed_dict[folder] = pickle.load(open(os.path.join(definitions.DATA_DIR, 'emissions_dict.pkl'), 'rb'))

..\functions\speed.py:48: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:50: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:51: RuntimeWarning:

invalid value encountered in greater



func:bin_2D_average took: 6.2380149364471436 sec


..\functions\speed.py:48: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:50: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:51: RuntimeWarning:

invalid value encountered in greater



func:bin_2D_average took: 5.335188388824463 sec


..\functions\speed.py:48: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:50: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:51: RuntimeWarning:

invalid value encountered in greater



func:bin_2D_average took: 6.620086193084717 sec


..\functions\speed.py:48: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:50: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:51: RuntimeWarning:

invalid value encountered in greater



func:bin_2D_average took: 7.091623544692993 sec
func:bin_2D_average took: 7.38750696182251 sec


..\functions\speed.py:48: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:50: RuntimeWarning:

invalid value encountered in greater

..\functions\speed.py:51: RuntimeWarning:

invalid value encountered in greater



In [86]:
plot_radius = 5
mapbox_key = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"
colorscale = [
              [0.0, "rgba(0, 255, 204, 1)"],
              [0.2, "rgb(0, 255, 51)"],
              [0.4, "rgb(204, 255, 0)"],
              [0.6, "rgb(255, 204, 51)"],
              [0.8, "rgb(255, 102, 51)"],
              [1.0, "rgb(204,0,0)"],
            ]
# colorscale = [
#             [0.0, "rgb(204,0,0)"],
#             [0.2, "rgb(255, 102, 51)"],
#             [0.4, "rgb(255, 204, 51)"],
#             [0.6, "rgb(204, 255, 0)"],
#             [0.8, "rgb(0, 255, 51)"],
#             [1, "rgba(0, 255, 204, 1)"],
#             ]

color_axis = dict(
            cmin=0,
            cmax=None,
            showscale=True,
            colorscale=colorscale,
            colorbar=dict(
                outlinecolor="black",
                outlinewidth=2,
                ticks="outside",
                # tickfont=pt.font_dict,
                tickvals=None, # tickvalues,
                ticktext=None,
                title=f"[mph]<br /> <br />",
                tickformat="e",
                # titlefont=pt.font_dict,
            )
)

In [87]:
plot_time_list = [['2020-02-10', pd.to_datetime('2020-02-10T07:30:00')], ['2020-02-11', pd.to_datetime('2020-02-11T07:30:00')], ['2020-02-12', pd.to_datetime('2020-02-12T07:30:00')], 
                 ['2020-02-13', pd.to_datetime('2020-02-13T07:30:00')], ['2020-02-14', pd.to_datetime('2020-02-14T07:30:00')]]

num_plots = len(plot_time_list)
fig = make_subplots(rows=1, cols=num_plots,
                    specs=[[{'type': 'mapbox', }] * num_plots],
                    subplot_titles=[str(time[1]) for time in plot_time_list],
                    column_widths=[(1 - 0.1 * (num_plots / 2)) / num_plots]*num_plots,
                    horizontal_spacing=0.05,)

mapbox_dict = {'mapbox': dict(accesstoken=mapbox_key,
                                bearing=0,
                                style='mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte',
                                center=go.layout.mapbox.Center(
                                    lat=33.126275,
                                    lon=-87.54891
                                ),
                                pitch=0,
                                zoom=14.45),}

max_speed = 60 # np.log10(60)
min_speed = 0  # np.log10(5)

max_value = -20
for i, time in enumerate(plot_time_list):
    
    z_data = np.array(average_speed_dict[time[0]][time[1]]['data'])  # np.log10(average_speed_dict[time[0]][time[1]]['data'])
    lat_edges = np.array(average_speed_dict[time[0]][time[1]]['lat'])
    lon_edges = np.array(average_speed_dict[time[0]][time[1]]['lon'])
    max_value = average_speed_dict[time[0]][time[1]]['max_value'] if average_speed_dict[time[0]][time[1]]['max_value'] > max_value else max_value
    
    fig.add_trace(go.Densitymapbox(lat=lat_edges[np.nonzero((z_data > min_speed) & (z_data < max_speed))],
                                    lon=lon_edges[np.nonzero((z_data > min_speed) & (z_data < max_speed))],
                                    z=max_speed - z_data[np.nonzero((z_data > min_speed) & (z_data < max_speed))],
                                    customdata=z_data[np.nonzero((z_data > min_speed) & (z_data < max_speed))],
                                   # customdata=summed_array[0][2],
                                    hovertemplate='%{customdata}', #if sum else None,
                                    radius=plot_radius,
                                    #hoverinfo='z', # z_data[np.nonzero((z_data > color_axis['cmin']) & (z_data < color_axis['cmax']))],
                                    coloraxis="coloraxis",
                                    reversescale=True
                                    ),
                    row=1,
                    col=i+1)
    
    mapbox_dict['mapbox'+ str(i+2)] =  mapbox_dict['mapbox']
    
# color_axis['cmax'] = math.ceil(max_value * 1000) / 1000


len_of_max = len(str(color_axis['cmax']))
# tickvalues = np.around(np.linspace(0, np.round(color_axis['cmax'], len_of_max-1), 7), 3)
tickvalues = np.around(np.linspace(min_speed, max_speed, 7), 3)
ticktext = [str(max_speed - val) for val in tickvalues]

color_axis['cmax'] = max_speed # np.log10(60)
color_axis['cmin'] = min_speed
color_axis['colorbar']['tickvals'] = tickvalues
color_axis['colorbar']['ticktext'] = ticktext

In [53]:
ticktext

['59.0', '49.167', '39.333', '29.5', '19.667', '9.832999999999998', '0.0']

In [88]:
fig.update_layout(mapbox_dict)

fig.update_layout(
    title={"text": "Average Speed", 'yanchor': 'middle', 'xanchor': 'auto'},
    hovermode='closest',
    coloraxis=color_axis
)

fig.show('browser')
pio.write_json(fig, os.path.join(definitions.ROOT, 'raw_plots', 'mapbox.plotly'))